# Sample query generator
- This is for generating sample queries to easy understand each retrieval method. 

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Creatr Bedrock client

In [6]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [7]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Load LLM

### LLM 로딩 (Claude-v3-sonnet)

In [8]:
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [20]:
llm_text = ChatBedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Sonnet"),
    client=boto3_bedrock,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm_text

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fd1d2bdb760>], client=<botocore.client.BedrockRuntime object at 0x7fd1c2b2c190>, model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']}, streaming=True)

## 3. LangChainOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [21]:
import boto3
from utils.ssm import parameter_store

In [22]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [23]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [24]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [25]:
index_name = opensearch_user_password = pm.get_params(
    key="opensearch_index_name",
    enc=True
)

print (f'index_name: {index_name}')

index_name: summit-workshop-index-unstructured-pymupdf-llama


In [26]:
index_name = "summit-workshop-index"

### OpenSearch Client 생성

In [27]:
from utils.opensearch import opensearch_utils

In [28]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 4. Query generation

In [72]:
from termcolor import colored
from langchain_core.tracers import ConsoleCallbackHandler
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

### prompt

In [78]:
system_prompt = '''
                You are a master answer bot designed to create sample questions for someone to easily understand rerieval methods.
                I'm going to give you the description of retrieval method and contexts.
                Read the contexts carefully, This is because you will be asked to create questions that given retrieval methods can answer well.
                Here is the descriptions of retrieval methods respectively.
                <description>
                - lexical search: Lexical search relies on matching the literal or surface-level representation of words and phrases in the query with the content in the search index. It primarily involves looking for exact matches of keywords without considering the broader meaning or context of the words. Lexical search can be precise but might lead to missed results if the exact terms aren’t present in the document. If a user searches for “apple pie recipes,” a lexical search engine returns results that specifically contain the words “apple,” “pie,” and “recipes.”
                - semantic search: Semantic search focuses on understanding the meaning of the query and the context of the information rather than just matching keywords. It uses natural language processing (NLP), machine learning (ML), and other advanced techniques to comprehend the intent behind a user’s query. Semantic search considers synonyms, related concepts, and the relationship between words to provide more contextually relevant results. If a user searches for “healthy recipes,” a semantic search engine might also include recipes that use terms like “nutritious meals” or “wholesome cooking.”
                - rag fusion: RAG-Fusion is able to provide accurate and comprehensive answers due to the generated queries contextualizing the original query from various perspectives. However, some answers strayed off topic when the generated queries' relevance to the original query is insufficient. 
                - hyde: Occasionally, when faced with a question that lacks specificity or lacks easily identifiable elements to derive an answer from a given context, it can be quite challenging. HyDE uses a Language Learning Model, like Claude, to create a theoretical document when responding to a query, as opposed to using the query and its computed vector to directly seek in the vector database. Rather than seeking embedding similarity for questions or queries, it focuses on answer-to-answer embedding similarity.
                </description>
                '''
human_prompt = """
               Here is the contexts as texts: <contexts>{contexts}</contexts>

               First, find a few paragraphs or sentences from the contexts that can revel difference among retrieval methods well. 
               Then, create the question and its answer as much as you can.

               Skip the preamble and go straight into the answer and reason why you create.
               Don't insert any XML tag such as <contexts> and </contexts> when answering.
               Answer in Korean.

               """
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)
prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

### chain

In [79]:
chain = prompt | llm_text | StrOutputParser()

verbose = False
invoke_args = {
    "contexts": "상록수(3H) 중점 교육활동 수(秀),Head 록(綠),Health 상(常), Heart 중점 교육 소통하는 공동체 하나되는 세계시민교육 나눔, 배려, 함께하는 감성이 자라는 예술교육 디지털 시민 역량 강화 독서인문교육 놀이와 쉼이 있는 교육 디양한 진로 교육 운영 체험중심 생태환경교육 인성교육 수(秀),Head 록(綠),Health 상(常), Heart 소통하는 공동체 하나되는 세계시민교육 나눔, 배려, 함께하는 감성이 자라는 예술교육 디지털 시민 역량 강화 독서인문교육 놀이와 쉼이 있는 교육 디양한 진로 교육 운영 체험중심 생태환경교육 인성교육 수(秀),Head 록(綠),Health 상(常), Heart 소통하는 공동체 하나되는 세계시민교육 나눔, 배려, 함께하는 감성이 자라는 예술교육 디지털 시민 역량 강화 독서인문교육 놀이와 쉼이 있는 교육 디양한 진로 교육 운영 체험중심 생태환경교육 인성교육 수(秀),Head 록(綠),Health 상(常), Heart 소통하는 공동체 하나되는 세계시민교육 나눔, 배려, 함께하는 감성이 자라는 예술교육 디지털 시민 역량 강화 독서인문교육 놀이와 쉼이 있는 교육 디양한 진로 교육 운영 체험중심 생태환경교육 인성교육 © © © 62 3 © 3 62 © . © . © 참여와 즐거운 배움의 열기로 신나는 학교 학교상 참여와 즐거운 배움의 열기로 신나는 학교 교사상 학생상 학부모상 큰 꿈을 키워가는 어린이 건강한 인성 감성을 가진 어린이 미래를 키워주는 교사 사랑과 열정으로 가르치는 교사 신뢰받는 교사 참여하는 학부모 믿음을 갖는 학부모 학교상 교사상 학생상 학부모상 1 - 상(常) 1. 바른 인성으로 존중과 나눔을 실천하는 자주적인 어린이"
}
response = chain.invoke(
    invoke_args,
    config={'callbacks': [ConsoleCallbackHandler()]} if verbose else {}
)


안녕하세요. 주어진 문맥을 바탕으로 다음과 같은 질문과 답변을 제시해 볼 수 있겠습니다.

1. 질문: 상록수 학교의 중점 교육활동 중 '디지털 시민 역량 강화' 교육과 관련된 문장은 무엇입니까?
답변: "디지털 시민 역량 강화"

이 질문은 정확한 키워드 매칭을 필요로 하므로 어휘 검색에 적합합니다.

2. 질문: 상록수 학교의 중점 교육활동에서 '함께하는 감성'을 기르기 위한 교육은 무엇인가요?  
답변: "나눔, 배려, 함께하는 감성이 자라는 예술교육"

이 질문은 '함께하는 감성'이라는 구체적인 단어는 없지만, 의미를 파악해야 하므로 의미 검색이 적절합니다.

3. 질문: 상록수 학교는 어떤 방식으로 학생들의 진로교육을 제공하고 있나요?
답변: "디양한 진로 교육 운영"

이 질문에 대한 답변을 찾기 위해서는 "디양한 진로 교육"이라는 구절의 맥락을 이해해야 하므로, RAG 퓨전이 도움이 될 수 있습니다.

4. 질문: 상록수 학교에서 제공하는 교육 중 학생들의 마음가짐과 관련된 교육은 무엇인가요?  
답변: 문맥에는 학생들의 마음가짐과 직접적으로 연관된 내용이 없습니다. 이 경우 HyDE 방식으로 새로운 이론적 문서를 생성하여 답변할 수 있습니다.

이렇게 다양한 질문을 만들어 봄으로써 각 검색 방법의 장단점을 파악할 수 있습니다.

In [ ]:
is_done = False
fetched_count = 0
loop_count = 0
offset = 0
count = 0
limit = docs_per_request = 5

query = {"query": {"match_all": {}}}

while not is_done:
    try:
        fetched_count += 1
        fetched_docs = os_client.search(
            index=index_name,
            body=query,
            size=limit,
            from_=offset
        )
        fetched_count = 0
    except Exception as e:
        if fetched_count == 3:
            print("Terminating script as connection is timeout more than 3 times.")
            break
        print ("{} Couldn't get records trying again for limit:{} and offset:{}".format(e, limit, offset))
        continue

    fetched_docs = fetched_docs["hits"]["hits"]
    loop_count += 1

    for index, doc in enumerate(fetched_docs):
        # Process the doc here.
        doc_id = doc["_id"]
        doc_text = doc["_source"]["text"]
        doc_metadata = doc["_source"]["metadata"]
        
        if doc_metadata["family_tree"] == "parent":
        
            print (colored(f'DOC ID: {doc_id}', "green"))
            print (colored(f'TEXT: {doc_text}', "blue"))
            print (colored(f'FAMILY_TREE: {doc_metadata["family_tree"]}', "red"))
            
            invoke_args = {
                "contexts": doc_text
            }

            response = chain.invoke(
                invoke_args,
                config={'callbacks': [ConsoleCallbackHandler()]} if verbose else {}
            )

#         questions = llm_chain_retriever.predict(context=doc_text, num_questions_per_chunk=str(num_questions_per_chunk))
#         #print (questions)
#         questions = questions.split("\n\n-")
#         if len(questions) <= num_questions_per_chunk + 1:

#             if len(questions) == num_questions_per_chunk:
#                 questions = list(map(lambda x:x.strip(), questions))
#             else:
#                 questions = list(map(lambda x:x.strip(), questions[1:]))
#             for q in questions:
#                 answer = llm_chain_generation.predict(question=q, context=doc_text)
#                 answer = answer.strip()
#                 #answer = answer[1:-1].strip()
#                 #print (colored(f'question: {q}', "green"))
#                 #print (colored(f'answer: {answer}', "blue"))
#                 gt.append([q, answer, doc_id, doc_text])
#         else:
#             print ("err")
#             print (questions)

#         #print ("==")
#     #break
    #if loop_count == 3: break
    offset += docs_per_request
    if len(fetched_docs) < docs_per_request:
        print("This is last batch.")
        is_done = True

    print("batch {} completed".format(count))

DOC ID: 9be52644-57c2-4525-adc1-f0d2e0e61ace
TEXT: 상록수(3H) 중점 교육활동 수(秀),Head 록(綠),Health 상(常), Heart 중점 교육 소통하는 공동체 하나되는 세계시민교육 나눔, 배려, 함께하는 감성이 자라는 예술교육 디지털 시민 역량 강화 독서인문교육 놀이와 쉼이 있는 교육 디양한 진로 교육 운영 체험중심 생태환경교육 인성교육 수(秀),Head 록(綠),Health 상(常), Heart 소통하는 공동체 하나되는 세계시민교육 나눔, 배려, 함께하는 감성이 자라는 예술교육 디지털 시민 역량 강화 독서인문교육 놀이와 쉼이 있는 교육 디양한 진로 교육 운영 체험중심 생태환경교육 인성교육 수(秀),Head 록(綠),Health 상(常), Heart 소통하는 공동체 하나되는 세계시민교육 나눔, 배려, 함께하는 감성이 자라는 예술교육 디지털 시민 역량 강화 독서인문교육 놀이와 쉼이 있는 교육 디양한 진로 교육 운영 체험중심 생태환경교육 인성교육 수(秀),Head 록(綠),Health 상(常), Heart 소통하는 공동체 하나되는 세계시민교육 나눔, 배려, 함께하는 감성이 자라는 예술교육 디지털 시민 역량 강화 독서인문교육 놀이와 쉼이 있는 교육 디양한 진로 교육 운영 체험중심 생태환경교육 인성교육 © © © 62 3 © 3 62 © . © . © 참여와 즐거운 배움의 열기로 신나는 학교 학교상 참여와 즐거운 배움의 열기로 신나는 학교 교사상 학생상 학부모상 큰 꿈을 키워가는 어린이 건강한 인성 감성을 가진 어린이 미래를 키워주는 교사 사랑과 열정으로 가르치는 교사 신뢰받는 교사 참여하는 학부모 믿음을 갖는 학부모 학교상 교사상 학생상 학부모상 1 - 상(常) 1. 바른 인성으로 존중과 나눔을 실천하는 자주적인 어린이
FAMILY_TREE: parent
1) 질문: 학교에서 "인성교육"에 대해 설명하고 있는 부분을 통해 각 검색 방식의 차이를 보여줄 수 있습니다. 어떤 질문을 만들 수 있을까요?

답변: 
- 어휘 